# Description

This notebook includes the necessary code to run the baseline solution for the collaborative filtering Kaggle competition for the CIL 2022 course.

You can run the baseline method in the 3 different modes available:

1.   with parameter tuning.
2.   with hard-coded parameter tuning result.
3.   with the original parameters given by the course staff.

Before you do, please run the entire "Preprocess" section. After that, choose one mode and run it. You can then create a submission file by running the section "Create submission file".


# Preprocess

### Download data from Kaggle

In [1]:
!pip install kaggle

!mkdir ~/.kaggle

import json

kaggle_username = "yuvalnis" #@param {type:"string"}
kaggle_api_key = "1800d5a286834f0416c338c7bd7f6dee" #@param {type:"string"}

assert len(kaggle_username) > 0 and len(kaggle_api_key) > 0

api_token = {"username": kaggle_username,"key": kaggle_api_key}

with open('kaggle.json', 'w') as file:
    json.dump(api_token, file)

!mv kaggle.json ~/.kaggle/kaggle.json

!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c cil-collaborative-filtering-2022

!unzip -n cil-collaborative-filtering-2022.zip

The syntax of the command is incorrect.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "C:\Users\yuval\anaconda3\envs\cil-runtime-terror\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\yuval\anaconda3\envs\cil-runtime-terror\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\yuval\anaconda3\envs\cil-runtime-terror\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "C:\Users\yuval\anaconda3\envs\cil-runtime-terror\lib\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "C:\Users\yuval\anaconda3\envs\cil-runtime-terror\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located 

## Imports

In [7]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from typing import Tuple, List

## Declare global parameters of data

In [8]:
total_num_users = 10000
total_num_movies = 1000

## Data parsing helper function declarations

In [10]:
def parse_csv(csv_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """
  Extract the arrays of user indices, item indices and ratings listed in a .csv file

  :param csv_path: path to .csv file to read from
  :return: 3 arrays containing the users indices, the item indices and the observed ratings in order  
  """
  df = pd.read_csv(csv_path)
  # extract user and item indices from the Id label in the dataframe
  df = df.join(df.Id.str.extract(r"r(?P<User>\d+)_c(?P<Item>\d+)").astype(int) - 1)
  # extract user, item and prediction triplets from dataframe
  users = df.User.values
  items = df.Item.values
  preds = df.Prediction.values
  return users, items, preds

def construct_ratings_matrix(
    users: np.ndarray,
    items: np.ndarray,
    ratings: np.ndarray,
    n_users: int,
    n_items: int
) -> Tuple[np.ndarray, np.ndarray]:
  """
  Constructs the ratings matrix with NaN values where no rating was observed

  :param users: array of user indices
  :param items: array of item indices
  :param predictions: array of ratings per user-item pair
  :param n_users: total number of users
  :param n_items: total number of items
  :return: the ratings matrix and the observed ratings mask
  """
  ratings_matrix = np.zeros((n_users, n_items))
  observed_mask = np.full((n_users, n_items), fill_value=False)
  for r, c, v in zip(users, items, ratings):
    observed_mask[r, c] = True
    ratings_matrix[r][c] = v

  ratings_matrix[~observed_mask] = np.nan

  return ratings_matrix, observed_mask

## Model function declarations

In [11]:
def KAA_splicer(covmat, selection_A):
  assert(selection_A.shape[0] == covmat.shape[0])
  assert(selection_A.shape[0] == covmat.shape[1])
  temp = covmat[:, selection_A]
  temp = temp[selection_A, :]
  return temp

def KXA_splicer(covmat, selection_A):
  assert(selection_A.shape[0] == covmat.shape[0])
  assert(selection_A.shape[0] == covmat.shape[1])
  temp = covmat[:, selection_A]
  temp = temp[np.logical_not(selection_A), :]
  return temp

def MUA_splicer(mu, selection_A):
  assert(selection_A.shape[0] == mu.shape[0])
  temp = mu[np.logical_not(selection_A)]
  return temp

def gauss_kernel(a, b, bandwidth):
  return np.exp(-(np.linalg.norm(a-b)**2)/bandwidth**2)

#X is the recommender system Matrix and entry_mask masks the existend entries
def gp_imputation(X, bandwidth, noise):
  #Xcombined = np.concatenate((X, Xtest))
  Xcombined = X
  maskedXcombined = np.ma.array(Xcombined, mask = np.logical_not(np.isnan(Xcombined)))
  mean = np.nanmean(Xcombined, axis = 0)
  cov = np.zeros((Xcombined.shape[1], Xcombined.shape[1]))
  notnan_mask = np.logical_not(np.isnan(Xcombined))
  for i in range(Xcombined.shape[1]):
    localmask = notnan_mask[:, i]
    #cov[i, i] = (np.matmul(Xcombined[localmask, i], Xcombined[localmask, i])**2+1)**2
    cov[i, i] = np.var(Xcombined[localmask, i])
    #cov[i, i] = 1.0
    for j in range(i + 1, Xcombined.shape[1]):
      localmask = notnan_mask[:, i] & notnan_mask[:, j]
      t1 = Xcombined[localmask, i]
      t2 = Xcombined[localmask, j]
      #temp = np.cov([t1, t2])
      #cov[i, j] = (np.matmul(t1, t2) + 1)**2
      #cov[i, j] = temp[0, 1]
      cov[i, j] = gauss_kernel(t1, t2, bandwidth)
      cov[j, i] = cov[i, j]
  Xcombined_new = np.copy(Xcombined)

  print(Xcombined.shape)

  for index, features in enumerate(Xcombined):
    selection_A = notnan_mask[index, :]
    mu = MUA_splicer(mean, selection_A)
    kxa = KXA_splicer(cov, selection_A)
    kaa = KAA_splicer(cov, selection_A)
    ya = Xcombined[index, selection_A]
    #print(mu.shape)
    #print(kxa.shape)
    #print(kaa.shape)
    #print(ya.T.shape)
    pred = mu + np.matmul(kxa, np.linalg.solve(kaa + np.identity(kaa.shape[0]) * noise**2, ya.T))
    #print(pred)
    #assert(False)
    for i in range(Xcombined.shape[1]):
      curr = 0
      if np.logical_not(notnan_mask[index, i]):
        Xcombined_new[index, i] = pred[curr]
        curr += 1
  #assert(not np.any(np.isnan(Xcombined_new)))
  return Xcombined_new


def normalize(data: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """
  Normalizes the input data matrix per item (column). NaN entries are disregarded and are set to zero.  

  :param data: matrix to normalize
  :return: the normalized input matrix, the column-wise mean and the column-wise standard deviation
  """
  # compute mean and std and normalized the matrix with NaN values
  mean = np.nanmean(data, axis=0, keepdims=True)
  std = np.nanstd(data, axis=0, keepdims=True)
  normed_data = (data - mean) / std
  # set the non-observed entries to 0
  normed_data = gp_imputation(normed_data, 0.1, 1.0)
  return normed_data, mean, std


def denormalize(data: np.ndarray, mean: np.ndarray, std: np.ndarray) -> np.ndarray:
  """
  Denormalizes the input data matrix per item (column) by performing the inverse operation of the normalize function  

  :param data: matrix to denormalize
  :param mean: the column-wise mean of the unnormalized matrix
  :param std: the column-wise std of the unnormalized matrix
  :return: the denormalized matrix
  """
  return (data * std) + mean


def SVD(A: np.ndarray, k: int) -> np.ndarray:
  """
  Computes the singular value decomposition of the input matrix, projected on
  the subspace defined by the k largest singular values, as a 3-tuple

  :param A: matrix to decompose
  :param k: number of the largest singular values
  :return: the projected singular value decomposition of A
  """
  assert(k <= min(A.shape)), "k should be no greater than the number of users or items."
  U, S, VT = np.linalg.svd(A, full_matrices=False)
  return U[:, :k], np.diag(S[:k]), VT[:k, :]


def RMSE(x: np.ndarray, y: np.ndarray, mask: np.ndarray) -> float:
  return np.sqrt(np.nansum(mask * (x - y) ** 2) / np.sum(mask))


def ALS(
    data: np.ndarray,
    mask: np.ndarray,
    U: np.ndarray,
    VT: np.ndarray,
    n_latent_factors: int,
    regularization_param: float,
    n_iterations: int
) -> np.ndarray:
  regularizer = regularization_param * np.eye(n_latent_factors)

  with tqdm(total=n_iterations * (np.sum(mask.shape))) as pbar:
    rmse_score = RMSE(data, U @ VT, mask)
    pbar.set_description(f"Initial RMSE score is {rmse_score:.4f}")
    for iter in range(n_iterations):
      for i, Ri in enumerate(mask):
        U[i] = np.linalg.solve(
            np.dot(VT, np.dot(np.diag(Ri), VT.T)) + regularizer,
            (np.dot(VT, np.dot(np.diag(Ri), data[i].T))).T
        )
        pbar.update(1)

      for j, Rj in enumerate(mask.T):
        VT[:,j] = np.linalg.solve(
            np.dot(U.T, np.dot(np.diag(Rj), U)) + regularizer,
            np.dot(U.T, np.dot(np.diag(Rj), data[:, j]))
        )
        pbar.update(1)

      rmse_score = RMSE(data, U @ VT, mask)
      pbar.set_description(f"At iteration #{iter + 1} the RMSE score is {rmse_score:.4f}")

  return U @ VT


def baseline(
    data: np.ndarray,
    mask: np.ndarray,
    n_latent_factors: int,
    regularization_param: float,
    n_iterations: int
) -> np.ndarray:
  """
  Runs the baseline (normalization, SVD, ALS and then denormalization) method

  :param data: the ratings matrix
  :param mask: the matrix of observed ratings
  :param n_latent_factors: the number of latent factor to use
  :param regularization_param: the regularizer (lambda) to use in the ALS steps
  :param n_iterations: the number of ALS iterations to do
  :return: the predicted ratings matrix
  """
  norm_data, data_mean, data_std = normalize(data)
  U, _, VT = SVD(norm_data, n_latent_factors)
  pred_ratings = ALS(norm_data, mask, U, VT, n_latent_factors, regularization_param, n_iterations)
  return denormalize(pred_ratings, data_mean, data_std)

# Setup baseline model parameters

## With parameter tuning

We tune the parameters of the baseline model using the grid search method over pairs of values for the number of latent factors used in SVD and ALS, and the regularization parameter used in ALS.

Split the data into train and test subsets: 

In [ ]:
users, items, preds = parse_csv('../data/data_train.csv')
# determine a random test set for the parameter tuning
train_users, test_users, train_items, test_items, train_preds, test_preds = \
  train_test_split(users, items, preds, test_size=0.1, random_state=42)
# construct ratings and masks for train and test sets
train_ratings, train_mask = construct_ratings_matrix(
    train_users, train_items, train_preds, total_num_users, total_num_movies
)
test_ratings, test_mask = construct_ratings_matrix(
    test_users, test_items, test_preds, total_num_users, total_num_movies
)

Declare ranges of parameters to tune:

In [ ]:
reg_param_vals = [0.1, 0.25, 0.5]
latent_dim_vals = range(2, 17, 2)
n_iterations = 20

Run a grid search algorithm to check the RMSE score on the validation subset given a parameter set:

In [ ]:
results: List[Tuple[float, int, float, float]] = list()
best_val_rmse = None
with tqdm(total=len(reg_param_vals) * len(latent_dim_vals)) as pbar:
  for i, reg in enumerate(reg_param_vals):
    for j, latent_dim in enumerate(latent_dim_vals):
      # compute predicted ratings with parameter pair
      pred_ratings = baseline(train_ratings, train_mask, latent_dim, reg, n_iterations)
      # compute and report RMSE train and test scores
      train_rmse = RMSE(pred_ratings, train_ratings, train_mask)
      test_rmse = RMSE(pred_ratings, test_ratings, test_mask)
      print(f"Regularizing parameter: {reg:.1f}, " \
            f"number of latent factors: {latent_dim}, " \
            f"train RMSE score: {train_rmse:.4f}, " \
            f"test RMSE score: {test_rmse:.4f}")
      # save RMSE scores in table
      results.append((reg, latent_dim, train_rmse, test_rmse))

      best_val_rmse = test_rmse if (best_val_rmse is None or test_rmse < best_val_rmse) else best_val_rmse
      pbar.update(1)
      pbar.set_description(f"(regularization param: {reg:.2f}, latent dim: {latent_dim}): minimal RMSE score on validation set is {best_val_rmse:.4f}")


Save results of the grid search to a .csv file for future reference:

In [ ]:
print(f"Saving the RMSE scores table to file...")
results_df = pd.DataFrame({
    "Regularization parameter": [reg_param for reg_param, _, _, _ in results],
    "Number of latent factors": [n_latent_factors for _, n_latent_factors, _, _ in results],
    "fit RMSE": [fit_rmse for _, _, fit_rmse, _ in results],
    "validation RMSE": [val_rmse for _, _, _, val_rmse in results]
})
results_df.to_csv(f"../results/baseline_param_tuning_results_reg.csv", index=False)
print(f"Done!")
print(results_df)

Extract the optimal parameters:

In [ ]:
regularization_param, n_latent_factors, _, min_val_rmse = min(results, key=(lambda t: t[3]))
print(f"The optimal regularization parameter is {regularization_param:.2f}")
print(f"The optimal number of latent factors is {n_latent_factors}")
print(f"The optimal validation RMSE is {min_val_rmse:.4f}")

## Hardcoded optimal parameters (skip parameter tuning)

Declare baseline parameters found after the parameter tuning algorithm:

In [ ]:
n_latent_factors = 3
regularization_param = 0.1
n_iterations = 20

## With original parameters

Declare baseline parameters used tp set the Kaggle competition baseline score:

In [12]:
n_latent_factors = 3
regularization_param = 0.1
n_iterations = 20

# Run baseline model on full dataset

In [13]:
users, items, preds = parse_csv('data_train.csv')
ratings, mask = construct_ratings_matrix(users, items, preds, total_num_users, total_num_movies)
pred_ratings = baseline(ratings, mask, n_latent_factors, regularization_param, n_iterations)

(10000, 1000)


  0%|          | 0/220000 [00:00<?, ?it/s]

# Create submission file

In [14]:
pred_users, pred_items, _ = parse_csv('sampleSubmission.csv')
df = pd.DataFrame({
    "Id": [f"r{r + 1}_c{c + 1}" for r, c in zip(pred_users, pred_items)],
    "Prediction": [pred_ratings[r, c] for r, c in zip(pred_users, pred_items)]
})
df.to_csv(f"baseline_gp_reg-{regularization_param:.2f}_k-{n_latent_factors}_iters-{n_iterations}_submission.csv", index=False)